# Tarefa 02 Módulo 05

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito. 
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
propostas = pd.read_csv('application_record.csv')
pg = pd.read_csv('pagamentos_largo.csv')

In [3]:
pg.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


In [4]:
# Consultando tamanho da base
pg.shape

(20937, 26)

In [5]:
# Buscando uma visão ordenada das colunas para tentar em algum momento ajudar na análise
rename_columns = {
    'mes_0': 'mes_00',
    'mes_1': 'mes_01',
    'mes_2': 'mes_02',
    'mes_3': 'mes_03',
    'mes_4': 'mes_04',
    'mes_5': 'mes_05',
    'mes_6': 'mes_06',
    'mes_7': 'mes_07',
    'mes_8': 'mes_08',
    'mes_9': 'mes_09'
}
pg.rename(columns=rename_columns, inplace=True)

In [6]:
# Consultando df com os meses na ordem crescente
pg[pg.columns.sort_values()]

,ID,mes_00,mes_01,mes_02,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,...,mes_15,mes_16,mes_17,mes_18,mes_19,mes_20,mes_21,mes_22,mes_23,mes_24
0,5001718,0,0,0,0,0,0,0,NaN,0,...,0,NaN,0,0,0,NaN,NaN,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,5001723,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5001726,0,0,0,0,0,0,C,C,C,...,C,C,C,C,C,C,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
20933,5150476,0,0,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20934,5150480,0,0,0,0,0,0,C,C,C,...,C,C,C,C,C,C,C,C,C,C
20935,5150482,0,0,0,0,0,0,0,0,0,...,C,C,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Consultando tipos e qtde de respostas para uma coluna
pg['mes_00'].value_counts()

mes_00
0    20302
C      550
1       78
4        2
5        2
2        2
3        1
Name: count, dtype: int64

In [8]:
# Script para substituir os valores com o método isin() e substituir por default
for coluna in pg.columns:
    pg[coluna] = np.where(pg[coluna].isin(['2', '3', '4', '5']), 'default', pg[coluna])

In [9]:
# Consultando resultados de um mês com resposta default
pg['mes_00'].value_counts()

mes_00
0          20302
C            550
1             78
default        7
Name: count, dtype: int64

In [10]:
# Consultando resultados de um mês com resposta default
pg['mes_02'].value_counts()

mes_02
0          17354
C           1516
1            430
default       48
Name: count, dtype: int64

In [11]:
pg

,ID,mes_00,mes_01,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
20933,5150476,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,0
20934,5150480,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C
20935,5150482,0,0,0,0,C,C,C,C,C,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [12]:
# Filtrando df com mes_00 = default
pg[pg['mes_00'] == 'default']

,ID,mes_00,mes_01,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09
2331,5011164,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,...,C,C,C,default,default,default,NaN,NaN,NaN,NaN
4744,5024071,default,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
6175,5033916,default,default,NaN,NaN,NaN,default,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9939,5061351,default,NaN,0,0,0,NaN,NaN,NaN,0,...,NaN,NaN,0,NaN,0,NaN,0,0,0,NaN
10470,5065348,default,0,C,C,C,C,C,C,C,...,C,C,C,default,C,C,C,C,C,C
14785,5105045,default,default,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
18139,5125708,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,default,default,NaN,NaN,NaN,NaN


In [13]:
pg[pg.columns]

,ID,mes_00,mes_01,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
20933,5150476,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,0
20934,5150480,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C
20935,5150482,0,0,0,0,C,C,C,C,C,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [14]:
# Percorrendo todas as linhas e colunas e adicionando info na nova variável
pg['default_control'] = pg[pg.columns].isin(['default']).sum(axis=1) > 0

In [15]:
pg

,ID,mes_00,mes_01,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_23,mes_24,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09,default_control
0,5001718,0,0,0,0,0,0,0,0,NaN,...,0,NaN,0,0,0,0,NaN,0,NaN,False
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,False
2,5001720,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,NaN,NaN,False
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,0,0,0,C,C,C,C,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,False
20933,5150476,0,0,0,0,0,0,0,0,0,...,NaN,NaN,0,0,0,0,0,0,0,False
20934,5150480,0,0,C,C,C,C,C,C,C,...,C,C,0,0,0,C,C,C,C,False
20935,5150482,0,0,0,0,C,C,C,C,C,...,NaN,NaN,0,0,0,0,0,0,0,False


In [16]:
# Verificando resultado com o script acima
pg['default_control'].value_counts()

default_control
False    20506
True       431
Name: count, dtype: int64

In [17]:
# Consultando df com filtro
pg[pg['default_control'] == True]

,ID,mes_00,mes_01,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_23,mes_24,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09,default_control
130,5002126,0,0,C,C,C,C,C,C,C,...,C,C,NaN,1,default,default,default,default,default,True
177,5002283,0,0,1,default,0,0,0,0,0,...,C,C,0,0,0,0,0,0,0,True
303,5002642,0,0,0,0,1,0,0,0,C,...,C,C,0,0,0,0,0,1,default,True
355,5002795,0,1,0,0,C,C,C,C,C,...,C,C,0,0,0,1,default,0,0,True
556,5003267,1,default,C,C,C,C,C,C,C,...,C,C,default,1,1,C,C,C,C,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20523,5149192,0,0,default,default,default,NaN,NaN,NaN,NaN,...,NaN,NaN,default,default,default,default,default,default,default,True
20734,5149698,0,0,NaN,NaN,NaN,default,default,default,0,...,NaN,NaN,0,NaN,0,1,NaN,default,0,True
20744,5149728,0,0,0,0,default,C,C,C,C,...,NaN,NaN,0,0,0,0,0,0,0,True
20745,5149730,0,0,0,0,default,C,C,C,C,...,NaN,NaN,0,0,0,0,0,0,0,True


In [18]:
pg.head()

,ID,mes_00,mes_01,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_23,mes_24,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09,default_control
0,5001718,0,0,0,0,0,0,0,0,NaN,...,0,NaN,0,0,0,0,NaN,0,NaN,False
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,False
2,5001720,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,NaN,NaN,False
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,0,0,0,C,C,C,C,False


In [19]:
propostas.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [20]:
pg.dtypes

ID                 object
mes_00             object
mes_01             object
mes_10             object
mes_11             object
mes_12             object
mes_13             object
mes_14             object
mes_15             object
mes_16             object
mes_17             object
mes_18             object
mes_19             object
mes_02             object
mes_20             object
mes_21             object
mes_22             object
mes_23             object
mes_24             object
mes_03             object
mes_04             object
mes_05             object
mes_06             object
mes_07             object
mes_08             object
mes_09             object
default_control      bool
dtype: object

In [21]:
propostas.dtypes

ID                       int64
CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_MOBIL               int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
FLAG_EMAIL               int64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
dtype: object

In [22]:
# Alterando tipo da variável para conseguir realizar o JOIN
propostas['ID'] = propostas['ID'].astype(str)

In [23]:
propostas.dtypes

ID                      object
CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_MOBIL               int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
FLAG_EMAIL               int64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
dtype: object

In [24]:
# Right join
pg.merge(propostas, on = 'ID', how ='right')

,ID,mes_00,mes_01,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [25]:
# Left join
pg.merge(propostas, on = 'ID', how ='left')

,ID,mes_00,mes_01,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5001720,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,C,C,C,C,C,C,C,C,C,...,Civil marriage,House / apartment,-10886.0,-1315.0,1.0,1.0,0.0,0.0,Laborers,2.0
20933,5150476,0,0,0,0,0,0,0,0,0,...,Civil marriage,House / apartment,-10886.0,-1315.0,1.0,1.0,0.0,0.0,Laborers,2.0
20934,5150480,0,0,C,C,C,C,C,C,C,...,Married,House / apartment,-10808.0,-1739.0,1.0,0.0,0.0,0.0,Core staff,3.0
20935,5150482,0,0,0,0,C,C,C,C,C,...,Married,House / apartment,-10808.0,-1739.0,1.0,0.0,0.0,0.0,Core staff,3.0


In [26]:
# Right join
propostas.merge(pg, on = 'ID', how='right')

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,mes_23,mes_24,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09,default_control
0,5001718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,NaN,0,0,0,0,NaN,0,NaN,False
1,5001719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,C,C,False
2,5001720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,False
3,5001723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,NaN,NaN,False
4,5001726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,0,0,0,C,C,C,C,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20932,5150475,M,Y,Y,0.0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,...,C,C,C,C,C,C,C,C,C,False
20933,5150476,M,Y,Y,0.0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,...,NaN,NaN,0,0,0,0,0,0,0,False
20934,5150480,F,Y,Y,1.0,157500.0,Working,Secondary / secondary special,Married,House / apartment,...,C,C,0,0,0,C,C,C,C,False
20935,5150482,F,Y,Y,1.0,157500.0,Working,Secondary / secondary special,Married,House / apartment,...,NaN,NaN,0,0,0,0,0,0,0,False


In [27]:
# Adicionando filtro de maus pagadores em um novo dataframe
pg_maus = pg[pg['default_control'] == True]

In [28]:
# Right join com os maus pagadores
propostas.merge(pg_maus, on = 'ID', how = 'right')

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,mes_23,mes_24,mes_03,mes_04,mes_05,mes_06,mes_07,mes_08,mes_09,default_control
0,5002126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,NaN,1,default,default,default,default,default,True
1,5002283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,0,0,0,0,0,0,0,True
2,5002642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,0,0,0,0,0,1,default,True
3,5002795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,0,0,0,1,default,0,0,True
4,5003267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,C,default,1,1,C,C,C,C,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,5149192,M,Y,N,1.0,450000.0,Working,Higher education,Married,House / apartment,...,NaN,NaN,default,default,default,default,default,default,default,True
427,5149698,M,N,Y,0.0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,...,NaN,NaN,0,NaN,0,1,NaN,default,0,True
428,5149728,F,Y,Y,0.0,90000.0,Working,Secondary / secondary special,Married,House / apartment,...,NaN,NaN,0,0,0,0,0,0,0,True
429,5149730,F,Y,Y,0.0,90000.0,Working,Secondary / secondary special,Married,House / apartment,...,NaN,NaN,0,0,0,0,0,0,0,True


In [29]:
# Contagem dos valores do default
pg_true = (pg['default_control'] == True).sum()
pg_true

431